# LlamaIndex API Tutorial

## Introduction

LlamaIndex is a powerful data framework for connecting custom data sources to large language models (LLMs). It provides essential tools for building LLM-powered applications over your private data, focusing on:

- Data ingestion and connectivity
- Data structuring and indexing
- Retrieval-augmented generation (RAG)
- Structured analytics
- Knowledge graph construction
- Multi-agent workflows

This document provides a concise overview of LlamaIndex's core APIs and their usage patterns.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# First, let's install LlamaIndex and its dependencies
# !pip install llama-index

## Imports

In [9]:
import logging
from llama_index.core.agent.workflow import AgentWorkflow, FunctionAgent
from llama_index.core.tools import FunctionTool
from llama_index.core.indices.property_graph import SimpleLLMPathExtractor
from llama_index.core import (
    SimpleDirectoryReader, 
    VectorStoreIndex, 
    PropertyGraphIndex,
    StorageContext,
    load_index_from_storage
)
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from dotenv import load_dotenv
import os
import nest_asyncio

# to run async code in notebook
nest_asyncio.apply()

### Settings Configuration

LlamaIndex uses a global `Settings` object to manage default behaviors. We also configure `OpenAI` as the LLM provider that configures which model to use and its parameters like temperature.

In [4]:
load_dotenv("devops/env/default.env")
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Configure default settings
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
Settings.llm = llm

print("Environment setup complete!")

Environment setup complete!


## Document Loading

The first step in any LlamaIndex workflow is loading your data. LlamaIndex provides a variety of document loaders (called "readers") to ingest data from different sources.

`SimpleDirectoryReader` is a document loader that reads files from a directory or specific file paths. When invoked with `load_data()`, it returns a list of `Document` objects containing the text content and metadata from your files

For this tutorial, we'll create a simple text file and load it.

In [5]:
# Create a sample text file
with open("sample_data.txt", "w") as f:
    f.write("""
    LlamaIndex (GPT Index) is a data framework for your LLM applications.
    
    It provides:
    1. Data connectors to ingest your existing data sources
    2. Ways to structure your data (indices, graphs, etc.) 
    3. Query interfaces that make LLMs smarter about your data
    4. Tools for evaluation, monitoring, and continual learning
    
    The primary value props are:
    * Building RAG applications
    * Structured analytics over your data
    * Knowledge graph construction
    * Multi-agent frameworks over your data
    """)

# Load the data
documents = SimpleDirectoryReader(input_files=["sample_data.txt"]).load_data()

print(f"Loaded {len(documents)} document(s)")
print(f"Document content preview: {documents[0].text[:100]}...")

Loaded 1 document(s)
Document content preview: 
    LlamaIndex (GPT Index) is a data framework for your LLM applications.
    
    It provides:
   ...


## Building a Simple Vector Index

Indices are the heart of LlamaIndex - they structure your data for efficient retrieval and interaction with LLMs.

`VectorStoreIndex.from_documents()` creates a vector index by:
- Converting document chunks to embeddings
- Storing these vectors for similarity search
- Maintaining references to the original text

The `as_query_engine()` method creates a query interface for your index that handles:
- Converting queries to embeddings
- Retrieving relevant document chunks
- Sending these chunks to the LLM with your query
- Generating coherent responses

For this tutorial, we'll use the default in-memory vector store.

In [6]:
# Create a vector index from the documents
vector_index = VectorStoreIndex.from_documents(documents)

# Create a query engine
query_engine = vector_index.as_query_engine()

# Ask a question
response = query_engine.query("What are the main value propositions of LlamaIndex?")

print(response)

The main value propositions of LlamaIndex are building RAG applications, structured analytics over your data, knowledge graph construction, and multi-agent frameworks over your data.


## Saving and Loading Indices

Processing large document collections can be time-consuming and computationally expensive. LlamaIndex allows you to persist indices to disk and load them later, saving significant processing time for production applications.

`storage_context.persist()` saves your index to disk, avoiding the need to reprocess documents each time. The index components saved include:
- Vector embeddings
- Document content
- Index metadata

`StorageContext.from_defaults()` and `load_index_from_storage()` load a previously saved index, recreating the in-memory structures needed for querying.

Let's see how to save our vector index to disk and load it later.

In [ ]:
# Create a storage context
if not os.path.exists("./storage"):
    os.makedirs("./storage")

# Save the index to disk
vector_index.storage_context.persist(persist_dir="./storage")

# Later, we can load the index from disk
storage_context = StorageContext.from_defaults(persist_dir="./storage")
loaded_index = load_index_from_storage(storage_context)

# Use the loaded index
loaded_query_engine = loaded_index.as_query_engine()
loaded_response = loaded_query_engine.query("What tools does LlamaIndex provide?")

print(loaded_response)

LlamaIndex provides tools for evaluation, monitoring, and continual learning.


## Creating a Property Graph Index

Beyond vector indices, LlamaIndex supports knowledge graphs through its `PropertyGraphIndex`. These graphs capture not just the content of your documents, but the relationships between entities mentioned in them.

`PropertyGraphIndex` creates a graph representation of your data by extracting:
- Entities (nodes): Entities or concepts (e.g., "LlamaIndex", "RAG", "Vector Database")
- Relationships (edges): Relationships between nodes (e.g., "LlamaIndex -> ENABLES -> RAG")
- Properties (attributes): Attributes attached to nodes and edges

`SimpleLLMPathExtractor` uses the LLM to identify subject-predicate-object triplets in your text, building a structured graph. The resulting knowledge graph enables more complex queries that leverage relationships between concepts.

Let's create a simple property graph from our documents.

In [10]:
# Create a graph index with a path extractor
graph_index = PropertyGraphIndex.from_documents(
    documents,
    kg_extractors=[SimpleLLMPathExtractor(llm=llm, max_paths_per_chunk=5)]
)

# Create a graph query engine
graph_query_engine = graph_index.as_query_engine()

# Query the graph
graph_response = graph_query_engine.query("What is the relationship between LlamaIndex and LLMs?")

print(graph_response)

LlamaIndex is a data framework for LLM applications.


## Using LlamaIndex Agents

LlamaIndex provides a powerful agent framework that enables autonomous and semi-autonomous interactions with your data. Agents can use tools, follow reasoning steps, and execute complex workflows.

`FunctionAgent` creates an autonomous assistant that can:
- Use tools via `FunctionTool`
- Follow a system prompt for behavior guidance
- Maintain context across interactions

`AgentWorkflow` orchestrates interaction flow, managing:
- Tool invocation
- Response generation
- State management

Let's create a simple agent that can search our knowledge base.

In [25]:
# Define a simple tool
def search_knowledge_base(query: str) -> str:
    """Search the knowledge base for information."""
    query_engine = vector_index.as_query_engine()
    return str(query_engine.query(query))

# Create a tool from the function
search_tool = FunctionTool.from_defaults(
    name="search_knowledge_base",
    description="Searches the knowledge base for relevant information",
    fn=search_knowledge_base
)

# Create an agent
agent = FunctionAgent(
    name="KnowledgeAgent",
    description="An agent that answers questions using a knowledge base",
    system_prompt=(
        "You are a helpful assistant that answers questions using a knowledge base. "
        "If you don't know the answer, say so."
    ),
    llm=llm,
    tools=[search_tool]
)

# Create an agent workflow
workflow = AgentWorkflow(agents=[agent], root_agent=agent.name)

# Run the agent
response = await workflow.run("What can I use LlamaIndex for?")

print(response.response.blocks[0].text)

LlamaIndex can be used for the following purposes:
1. Building RAG applications
2. Structured analytics over data
3. Knowledge graph construction
4. Multi-agent frameworks over data


## Advanced Query Techniques

LlamaIndex offers sophisticated querying capabilities beyond basic question-answering. These techniques allow for more precise information retrieval and enhanced response quality.

The `as_retriever()` method provides direct access to document retrieval without response synthesis. You can configure:
- `similarity_top_k`: Number of documents to retrieve
- Metadata filters
- Retrieval strategies

Let's explore a more advanced retrieval approach.

In [27]:
# Create a retriever with metadata filtering
retriever = vector_index.as_retriever(
    similarity_top_k=2,  # Retrieve top 2 most similar documents
)

# Retrieve documents
retrieved_nodes = retriever.retrieve("What are the value props of LlamaIndex?")

print(f"Retrieved {len(retrieved_nodes)} documents")
for i, node in enumerate(retrieved_nodes):
    print(f"Document {i+1} (Score: {node.score}): {node.text}...")

Retrieved 1 documents
Document 1 (Score: 0.8177321887934406): LlamaIndex (GPT Index) is a data framework for your LLM applications.
    
    It provides:
    1. Data connectors to ingest your existing data sources
    2. Ways to structure your data (indices, graphs, etc.) 
    3. Query interfaces that make LLMs smarter about your data
    4. Tools for evaluation, monitoring, and continual learning
    
    The primary value props are:
    * Building RAG applications
    * Structured analytics over your data
    * Knowledge graph construction
    * Multi-agent frameworks over your data...


After mastering these basics, you can explore:
- Custom data connectors
- Different index types
- Fine-tuning query parameters
- Multi-modal data processing
- Production deployment strategies